In [1]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
 

In [ ]:
# grab the paths to the input images in our dataset, then initialize
# out data list (which we'll soon populate)
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images("./face-clustering/dataset"))
#imagePaths = list(paths.list_images(args["dataset"]))
data = []

In [ ]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# load the input image and convert it from RGB (OpenCV ordering)
	# to dlib ordering (RGB)
	print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
	print(imagePath)
	image = cv2.imread(imagePath)
	rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    	# detect the (x, y)-coordinates of the bounding boxes
	# corresponding to each face in the input image
	boxes = face_recognition.face_locations(rgb,model="hog")
	# compute the facial embedding for the face
	encodings = face_recognition.face_encodings(rgb, boxes)
 
	# build a dictionary of the image path, bounding box location,
	# and facial encodings for the current image
	d = [{"imagePath": imagePath, "loc": box, "encoding": enc}
		for (box, enc) in zip(boxes, encodings)]
	data.extend(d)

# dump the facial encodings data to disk
print("[INFO] serializing encodings...")
f = open("encodings", "wb")
f.write(pickle.dumps(data))
f.close()

In [2]:
# import the necessary packages
from sklearn.cluster import DBSCAN
from imutils import build_montages
import numpy as np
import pickle
import cv2

In [3]:
# load the serialized face encodings + bounding box locations from
# disk, then extract the set of encodings to so we can cluster on
# them
print("[INFO] loading encodings...")
data = pickle.loads(open("encodings", "rb").read())
data = np.array(data)
encodings = [d["encoding"] for d in data]

[INFO] loading encodings...


In [4]:
# cluster the embeddings
print("[INFO] clustering...")
clt = DBSCAN(metric="euclidean")
clt.fit(encodings)
 
# determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

[INFO] clustering...
[INFO] # unique faces: 4


In [5]:
import os
# loop over the unique face integers
for labelID in labelIDs:
	# find all indexes into the `data` array that belong to the
	# current label ID, then randomly sample a maximum of 25 indexes
	# from the set
	print("[INFO] faces for face ID: {}".format(labelID))
	idxs = np.where(clt.labels_ == labelID)[0]
	idxs = np.random.choice(idxs, size=min(25, len(idxs)),
		replace=False)
 
	# initialize the list of faces to include in the montage
	faces = []
    	# loop over the sampled indexes
	for i in idxs:
		# load the input image and extract the face ROI
		image = cv2.imread(data[i]["imagePath"])
		(top, right, bottom, left) = data[i]["loc"]
		face = image[top:bottom, left:right]
 
		# force resize the face ROI to 96x96 and then add it to the
		# faces montage list
		face = cv2.resize(face, (128, 128))
		request_number = labelID if labelID != -1 else "Unknown"
        # base dir
		_dir = "./face-clustering/dataset"
		

        # create dynamic name, like "D:\Current Download\Attachment82673"
		_dir = os.path.join(_dir, 'Person%s' % request_number)

        # create 'dynamic' dir, if it does not exist
		if not os.path.exists(_dir):
			os.makedirs(_dir)
		file_name = _dir + "/"+str(i)+".jpg"
		cv2.imwrite(file_name, face)
		
	#cv2.destroyAllWindows()    


      

[INFO] faces for face ID: -1
[INFO] faces for face ID: 0
[INFO] faces for face ID: 1
[INFO] faces for face ID: 2
[INFO] faces for face ID: 3


In [8]:
cv2.destroyAllWindows()

In [10]:
print(_dir)

./face-clustering/dataset/00000049.jpg


In [11]:
print(type(_dir))

<class 'str'>
